# Statistical Natural Language Processing (WS 20/21)
## Exercise Sheet 2 - Manuel Hettich

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

# non-zero probability
eps = 1e-40

In [2]:
'''
This function can be used for importing the corpus.
Parameters: path_to_file: string; path to the file containing the corpus
Returns: list of list; the second layer list contains tuples (token,label);
'''
def import_corpus(path_to_file):
    sentences = []
    sentence = []
    f = open(path_to_file)

    while True:
        line = f.readline()
        if not line: break

        line = line.strip()
        if len(line) == 0:
            sentences.append(sentence)
            sentence = []
            continue

        parts = line.split(' ')
        sentence.append((parts[0], parts[-1]))

    f.close()
    return sentences

### Exercise 1

In [3]:
# Setup internal state
sentences = import_corpus("corpus_ner.txt")

token_vocabulary = Counter()    # set of emission symbols K
tag_vocabulary = Counter()      # set of states S (labels)

# Count all tokens and tags in the corpus
for sentence in sentences:
    for token, tag in sentence:
        token_vocabulary.update([token])
        tag_vocabulary.update([tag])

#print(sum([1 for value in token_vocabulary.values() if value == 1]), len(token_vocabulary))
#print(tag_vocabulary)

UNKNOWN_TOKEN = "<unknown>"

def preprocess(sentence, token_counter):
    processed_sentence = []

    for token, tag in sentence:
        token_count = token_counter.get(token, 0)

        if token_count <= 1:
            token = UNKNOWN_TOKEN

        processed_sentence.append([token, tag])
    return processed_sentence

#sentences[0], "\n", preprocess(sentences[0], token_vocabulary)

In [4]:
'''
Implement the probability distribution of the initial states.
Parameters:	state: string
            internal_representation: data structure representing the parameterization of this probability distribution;
                this data structure is returned by the function estimate_initial_state_probabilities
Returns: float; initial probability of the given state
'''
def initial_state_probabilities(state, internal_representation):
    return internal_representation.get(state, eps)

In [5]:
'''
Implement the matrix of transition probabilities.
Parameters:	from_state: string;
            to_state: string;
            internal_representation: data structure representing the parameterization of the matrix of transition probabilities;
                this data structure is returned by the function estimate_transition_probabilities
Returns: float; probability of transition from_state -> to_state
'''
def transition_probabilities(from_state, to_state, internal_representation):
    return internal_representation.get(from_state, {}).get(to_state, eps)

In [6]:
'''
Implement the matrix of emmision probabilities.
Parameters:	state: string;
            emission_symbol: string;
            internal_representation: data structure representing the parameterization of the matrix of emission probabilities;
                this data structure is returned by the function estimate_emission_probabilities
Returns: float; emission probability of the symbol emission_symbol if the current state is state
'''
def emission_probabilities(state, emission_symbol, internal_representation):
    return internal_representation.get(state, {}).get(emission_symbol, eps)

In [7]:
'''
Implement a function for estimating the parameters of the probability distribution of the initial states.
Parameters: corpus: list returned by the function import_corpus
Returns: data structure containing the parameters of the probability distribution of the initial states;
            use this data structure for the argument internal_representation of the function initial_state_probabilities
'''
def estimate_initial_state_probabilities(corpus):
    initial_states = Counter()    # set of states S (labels, tag vocabulary)
    p_initial_states = dict()

    # Count each initial state for each sentence in the corpus
    initial_states.update([sentence[0][1] for sentence in corpus])
    total_initial_states = len(corpus)

    # Calculate the initial probability for each state/tag in the corpus
    for state in initial_states:
        p_initial_states[state] = initial_states[state] / total_initial_states

    return p_initial_states

In [8]:
estimate_initial_state_probabilities(import_corpus("corpus_ner.txt"))

{'O': 0.6713460772600558,
 'B-LOC': 0.07984866587017124,
 'B-PER': 0.11210673038630029,
 'B-ORG': 0.08562325766626842,
 'B-MISC': 0.051075268817204304}

In [26]:
'''
Implement a function for estimating the parameters of the matrix of transition probabilities
Parameters: corpus: list returned by the function import_corpus
Returns: data structure containing the parameters of the matrix of transition probabilities;
            use this data structure for the argument internal_representation of the function transition_probabilities
'''
def estimate_transition_probabilities(corpus):
    states = set()
    num_state_transitions = dict()
    p_state_transitions = dict()

    for sentence in corpus:
        for trans_end_idx in range(1, len(sentence)):
            from_state = sentence[trans_end_idx - 1][1]
            to_state = sentence[trans_end_idx][1]
            # Create a new Counter for from_state if it was not seen before
            if from_state not in num_state_transitions:
                num_state_transitions[from_state] = Counter()
            # Count the current transition defined by their states
            num_state_transitions[from_state].update([to_state])
            # Keep set of all transitions updated
            states.add(from_state)

    # Calculate the transition probabilities by normalising
    for from_state in states:
        if from_state not in p_state_transitions:
                    p_state_transitions[from_state] = dict()
        for to_state in states:
            if num_state_transitions.get(from_state, {}).get(to_state, False):
                total_to_states = np.sum([count for count in num_state_transitions[from_state].values()])
                p_state_transitions[from_state][to_state] = num_state_transitions[from_state][to_state] / total_to_states

    return p_state_transitions

In [27]:
estimate_transition_probabilities(import_corpus("corpus_ner.txt"))

{'I-MISC': {'I-MISC': 0.25357710651828297,
  'B-LOC': 0.001589825119236884,
  'B-MISC': 0.00397456279809221,
  'O': 0.7329093799682035,
  'B-ORG': 0.0047694753577106515,
  'B-PER': 0.003179650238473768},
 'B-LOC': {'B-LOC': 0.0006681142475363287,
  'B-MISC': 0.004342742608986137,
  'O': 0.8287957240688157,
  'B-ORG': 0.0025054284282612325,
  'I-LOC': 0.16368799064640052},
 'I-PER': {'I-PER': 0.07029478458049887, 'O': 0.9297052154195011},
 'B-MISC': {'I-MISC': 0.21730103806228374,
  'B-LOC': 0.0025374855824682814,
  'B-MISC': 0.00784313725490196,
  'O': 0.7427912341407151,
  'B-ORG': 0.010380622837370242,
  'B-PER': 0.01914648212226067},
 'O': {'B-LOC': 0.028280890253261703,
  'B-MISC': 0.02051858348865256,
  'O': 0.9041881372656507,
  'B-ORG': 0.021000986733910754,
  'B-PER': 0.026011402258524285},
 'B-ORG': {'B-LOC': 0.0008406893652795292,
  'B-MISC': 0.0018915510718789407,
  'O': 0.587431693989071,
  'I-ORG': 0.40920554854981084,
  'B-PER': 0.0006305170239596469},
 'I-ORG': {'B-MISC'

In [30]:
'''
Implement a function for estimating the parameters of the matrix of emission probabilities
Parameters: corpus: list returned by the function import_corpus
Returns: data structure containing the parameters of the matrix of emission probabilities;
            use this data structure for the argument internal_representation of the function emission_probabilities
'''
def estimate_emission_probabilities(corpus):
    states = set()
    emissions = Counter()
    num_emissions = dict()
    p_emission = dict()

    # Count all tokens and tags in the corpus
    for sentence in corpus:
        for emission, state in sentence:
            if state not in num_emissions:
                num_emissions[state] = Counter()
            num_emissions[state].update([emission])
            emissions.update([emission])
            states.add(state)

    # Treat all emissions which only occur once as unknown
    for emission, count in emissions.items():
        if count <= 1:
            # Remove emission and replace by <unknown>
            for state in states:
                if emission in num_emissions[state]:
                    num_emissions[state].update(["<unknown>"])
                    del(num_emissions[state][emission])

    # Calculate probabilities for each emission by normalising
    for state in states:
        if state not in p_emission:
            p_emission[state] = dict()
        total_emissions_per_state = np.sum([count for count in num_emissions[state].values()])
        for emission in num_emissions[state]:
            p_emission[state][emission] = num_emissions[state][emission] / total_emissions_per_state

    return p_emission

In [32]:
p_emissions = estimate_emission_probabilities(import_corpus("corpus_ner.txt"))
p_emissions['O']['<unknown>']

0.03916260509004749

### Exercise 2

In [13]:
''''
Implement the Viterbi algorithm for computing the most likely state sequence given a sequence of observed symbols.
Parameters: observed_symbols: list of strings; the sequence of observed symbols
            initial_state_probabilities_parameters: data structure containing the parameters of the probability distribution of the initial states, returned by estimate_initial_state_probabilities
            transition_probabilities_parameters: data structure containing the parameters of the matrix of transition probabilities, returned by estimate_transition_probabilities
            emission_probabilities_parameters: data structure containing the parameters of the matrix of emission probabilities, returned by estimate_emission_probabilities
Returns: list of strings; the most likely state sequence
'''
def most_likely_state_sequence(observed_symbols,
                               initial_state_probabilities_parameters,
                               transition_probabilities_parameters,
                               emission_probabilities_parameters):
    pass

In [14]:
most_likely_state_sequence([],
                           estimate_initial_state_probabilities(import_corpus("corpus_ner.txt")),
                           estimate_transition_probabilities(import_corpus("corpus_ner.txt")),
                           estimate_emission_probabilities(import_corpus("corpus_ner.txt")))